## Recommendation of movies using user ratings

- This project aims to explore recommendation systems using Tensorflow's [`TFRS`](https://www.tensorflow.org/recommenders) library
- This project will also be expanded to provide recommendations for a personal movie club later on with better quality data

In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")
# check data 
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


### Split ratings/movies into movie titles and user id

In [2]:

ratings = ratings.map(lambda x: {
        "movie_title": x["movie_title"],
        "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

### Shuffle data 

In [3]:

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

### Unique user ids and movie titles

In [4]:

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

### Query tower 

In [5]:

embedding_dimension = 32
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
    vocabulary=unique_user_ids, mask_token=None),
  # additional embedding to account for unkown tokens
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### Candidate tower

In [6]:

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metrics

In [7]:

metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

### Loss 

In [8]:

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

### Full model

In [9]:

class MovielensModel(tfrs.Model):
  
  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # User features to user model
    user_embeddings = self.user_model(features["user_id"])
    # movie features to movie model with embeddings
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # Task method computes loss and metrics
    return self.task(user_embeddings, positive_movie_embeddings)

### Fitting

In [10]:

model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

### Shuffle, batch, cache

In [11]:

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

### Train model

In [12]:

model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 8s 342ms/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0039 - factorized_top_k/top_50_categorical_accuracy: 0.0444 - factorized_top_k/top_100_categorical_accuracy: 0.1071 - loss: 69885.0661 - regularization_loss: 0.0000e+00 - total_loss: 69885.0661
Epoch 2/3
10/10 [==============================] - 3s 306ms/step - factorized_top_k/top_1_categorical_accuracy: 9.3750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0108 - factorized_top_k/top_10_categorical_accuracy: 0.0250 - factorized_top_k/top_50_categorical_accuracy: 0.1403 - factorized_top_k/top_100_categorical_accuracy: 0.2633 - loss: 67523.3061 - regularization_loss: 0.0000e+00 - total_loss: 67523.3061
Epoch 3/3
10/10 [==============================] - 3s 310ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_ac

### Evaluate model

In [13]:

model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 2s 196ms/step - factorized_top_k/top_1_categorical_accuracy: 8.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0095 - factorized_top_k/top_10_categorical_accuracy: 0.0225 - factorized_top_k/top_50_categorical_accuracy: 0.1248 - factorized_top_k/top_100_categorical_accuracy: 0.2327 - loss: 31079.0618 - regularization_loss: 0.0000e+00 - total_loss: 31079.0618


{'factorized_top_k/top_1_categorical_accuracy': 0.0008500000112690032,
 'factorized_top_k/top_5_categorical_accuracy': 0.009499999694526196,
 'factorized_top_k/top_10_categorical_accuracy': 0.02250000089406967,
 'factorized_top_k/top_50_categorical_accuracy': 0.12475000321865082,
 'factorized_top_k/top_100_categorical_accuracy': 0.23274999856948853,
 'loss': 28244.7734375,
 'regularization_loss': 0,
 'total_loss': 28244.7734375}

### Make predictions

In [14]:

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

_, title = index(tf.constant(["56"]))
print(f"Recommendations for user 56: {title[0, :3]}")

Recommendations for user 56: [b"Pete's Dragon (1977)" b"Kid in King Arthur's Court, A (1995)"
 b'Santa Clause, The (1994)']
